## Imports

In [1]:
import pandas as pd
import re, unidecode, sqlalchemy as sa
from sqlalchemy.orm import Session
from sqlalchemy.dialects.postgresql import insert
from tqdm.auto import tqdm

from apps.ingestion.seed_and_ingest import Player, FootballNews, Base, get_engine

DB_URL = "postgresql+psycopg2://scout:scout@db:5432/scouting"
engine = sa.create_engine(DB_URL)

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Players Table

In [2]:
# total de jugadores
with engine.connect() as conn:
    n_players = conn.scalar(sa.text("SELECT COUNT(*) FROM players"))
print(f"Hay {n_players:,} jugadores")

Hay 18,590 jugadores


In [3]:
df_players = pd.read_sql_table("players", con=engine)

In [4]:
df_players[df_players["full_name"].str.contains("Cardoso")]

,id,full_name,age,nationality,position,club,team_logo,league,minutes,minutes_90s,...,challenge_tackles_pct,challenges_lost,blocks,blocked_shots,blocked_passes,interceptions,tackles_interceptions,clearances,errors,feature_vector
60,182,Johnny Cardoso,22,USA,MF,Betis,https://cdn.ssref.net/req/202507021/tlogo/fb/f...,La Liga,2144,23.8,...,38.5,32,27,12,15,48,103,64,1,"[1.4187087, 1.4165306, 0.75571555, 0.14780061,..."
4345,12167,Nathan Cardoso,30,BRA,DF,Cuiabá,https://cdn.ssref.net/req/202507021/tlogo/fb/f...,Brasileirao B,618,6.9,...,0.0,0,0,0,0,0,0,0,0,"[-0.2773078, -0.27393377, -0.46199718, 0.14780..."
8268,2569,Tiago Cardoso,18,LUX,GK,Gladbach,https://cdn.ssref.net/req/202507021/tlogo/fb/3...,Bundesliga,380,4.2,...,100.0,0,0,0,0,0,1,2,2,"[-0.5418241, -0.54400796, -0.46199718, -0.4901..."
10403,5933,Felipe Cardoso,22,BRA,MF,Vitória,https://cdn.ssref.net/req/202507021/tlogo/fb/3...,Brasileirao,0,0.0,...,0.0,0,0,0,0,0,0,0,0,"[-0.9641611, -0.96412337, -0.46199718, -0.4901..."
14074,11602,Miguel Filipe Nunes Cardoso,30,POR,MF,Kayserispor,https://cdn.ssref.net/req/202507021/tlogo/fb/1...,Turkiye Super Lig,2857,31.7,...,0.0,0,0,0,0,0,0,0,0,"[2.211146, 2.2067475, 2.7852366, 3.9754486, -0..."
15836,14333,Gonçalo Cardoso,23,POR,DF,Lokomotiv Sofia,https://cdn.ssref.net/req/202507021/tlogo/fb/2...,Bulgarian First League,45,0.5,...,0.0,0,0,0,0,0,0,0,0,"[-0.9141475, -0.91410965, -0.46199718, -0.4901..."


In [ ]:
df_players.info()

## Creating feature_vector Column

In [ ]:
from sqlalchemy import create_engine, text

# 1️⃣  Conexión -------------------------------------------------
engine = create_engine(
    "postgresql+psycopg2://scout:scout@db:5432/scouting",
    isolation_level="AUTOCOMMIT",  # evita tener que hacer COMMIT explícito
)

# 2️⃣  Parámetros que necesitas saber de antemano --------------
PLAYER_DIM = 43   # ← dimensión real de tu vector de jugador
LISTS      = 140   # ← granularidad del índice IVFFLAT

with engine.begin() as conn:
    # 3️⃣  Activa la extensión pgvector (idempotente) -----------
    conn.execute(text("CREATE EXTENSION IF NOT EXISTS vector;"))

    # 4️⃣  Añade la nueva columna vector(N) y migra los datos ---
    conn.execute(text(f"""
        ALTER TABLE players
          ADD COLUMN IF NOT EXISTS feature_tmp vector({PLAYER_DIM});

        UPDATE players
          SET feature_tmp = feature_vector::vector
          WHERE feature_tmp IS NULL;      -- evita tocar filas ya convertidas

        ALTER TABLE players
          DROP COLUMN IF EXISTS feature_vector;
        ALTER TABLE players
          RENAME COLUMN feature_tmp TO feature_vector;
    """))

    # 5️⃣  Crea (o recrea) el índice IVFFLAT --------------------
    conn.execute(text(f"""
        CREATE INDEX IF NOT EXISTS players_feature_vec_idx
        ON players USING ivfflat (feature_vector vector_cosine_ops)
        WITH (lists = {LISTS});
    """))

print("Migración a pgvector finalizada ✔️")

### Populate feature_vector Column

In [ ]:
FEATURE_COLS = [
    "minutes", "minutes_90s",
    "goals", "assists",
    "expected_goals", "expected_assists",
    "no_penalty_expected_goals_plus_expected_assists",
    "progressive_carries", "progressive_passes", "progressive_passes_received",
    "goals_per90", "assists_per90", "goals_assists_per90",
    "expected_goals_per90", "expected_assists_per90", "expected_goals_assists_per90",
    "gk_goals_against", "gk_pens_allowed", "gk_free_kick_goals_against",
    "gk_corner_kick_goals_against", "gk_own_goals_against",
    "gk_psxg", "gk_psnpxg_per_shot_on_target_against",
    "passes_completed", "passes", "passes_pct",
    "passes_progressive_distance", "passes_completed_long", "passes_long", "passes_pct_long",
    "tackles", "tackles_won", "challenge_tackles", "challenges",
    "challenge_tackles_pct", "challenges_lost",
    "blocks", "blocked_shots", "blocked_passes",
    "interceptions", "tackles_interceptions", "clearances", "errors"
]

df = pd.read_sql(
    f"SELECT id, {', '.join(FEATURE_COLS)} FROM players",
    engine
)

In [ ]:
df

#### Standarization feature_vector values

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
emb_matrix = scaler.fit_transform(df[FEATURE_COLS]).astype("float32")

In [ ]:
df["feature_vector"] = [vec.tolist() for vec in emb_matrix]

In [ ]:
df.info()

In [ ]:
from pgvector.sqlalchemy import Vector

with engine.begin() as conn:
    conn.execute(
        text("""
            UPDATE players
            SET feature_vector = :vec
            WHERE id = :pid
        """),
        [{"pid": pid, "vec": vec} for pid, vec in zip(df.id, df.feature_vector)]
    )

#### Checks

In [ ]:
check = pd.read_sql(
    "SELECT vector_dims(feature_vector) AS dim, COUNT(*) FROM players GROUP BY dim;",
    engine
)
assert check["dim"].iloc[0] == 43, "Alguna fila no tiene dimensión 43"
print(check)

In [ ]:
with engine.begin() as conn:
    conn.execute(text("ANALYZE players;"))
    conn.execute(text("SET ivfflat.probes = 10;")) 

In [ ]:
query = """
EXPLAIN (ANALYZE, BUFFERS)
SELECT * FROM players
ORDER BY feature_vector <=> (SELECT feature_vector FROM players WHERE id = 42)
LIMIT 15;
"""

In [ ]:
with engine.begin() as conn:
    conn.execute(text(query))

In [ ]:
df_plan = pd.read_sql(query, engine)
display(df_plan)

In [ ]:
df

## Football news Table validations 

In [ ]:
# primeras 5 noticias
query = """
SELECT *
FROM football_news
ORDER BY published_at DESC
LIMIT 5
"""
df = pd.read_sql(query, con=engine)
df

In [ ]:
df.summary[4]

## Player Id - News ID table creation

In [ ]:
from sqlalchemy import create_engine, text

# 1️⃣  Conexión -------------------------------------------------
engine = create_engine(
    "postgresql+psycopg2://scout:scout@db:5432/scouting",
    isolation_level="AUTOCOMMIT",  # evita tener que hacer COMMIT explícito
)

In [ ]:
query = """
CREATE TABLE IF NOT EXISTS player_news (
  player_id  INTEGER NOT NULL REFERENCES players(id) ON DELETE CASCADE,
  news_id    INTEGER NOT NULL REFERENCES football_news(id) ON DELETE CASCADE,
  PRIMARY KEY (player_id, news_id)
);
"""

query2 = """
CREATE INDEX IF NOT EXISTS player_news_player_idx
ON player_news(player_id);
"""

In [ ]:
with engine.begin() as conn:
    conn.execute(text(query))

    conn.execute(text(query2))

print("Tabla Creada finalizada ✔️")

In [ ]:
player_news = sa.Table(
    "player_news", Base.metadata,
    sa.Column("player_id", sa.Integer),
    sa.Column("news_id",   sa.Integer),
    sa.PrimaryKeyConstraint("player_id", "news_id")
)

In [ ]:
def normalize(text: str) -> str:
    """minúsculas y sin tildes/acentos para comparar."""
    return unidecode.unidecode(text).lower().strip()

with Session(engine) as sess:
    # ── 1. Diccionario {nombre normalizado: id} ────────────────────────
    players = sess.query(Player.id, Player.full_name).all()
    name_to_id = {normalize(p.full_name): p.id for p in players}

    # regex con todos los nombres (palabra completa)
    pattern = r"\b(" + "|".join(re.escape(n) for n in name_to_id) + r")\b"
    name_re = re.compile(pattern, re.I)

    # ── 2. Iteramos con barra de progreso sobre las noticias ───────────
    news_rows = sess.query(FootballNews).all()
    inserted = 0

    for news in tqdm(news_rows, desc="Vinculando noticias", unit="noticia"):
        # nombres encontrados en el artículo
        matches = {normalize(m.group(0)) for m in name_re.finditer(news.article_text)}

        for norm_name in matches:
            player_id = name_to_id.get(norm_name)
            if not player_id:
                continue  # nombre no está en la tabla players

            stmt = (
                insert(player_news)
                .values(player_id=player_id, news_id=news.id)
                .on_conflict_do_nothing()
            )
            sess.execute(stmt)
            inserted += 1

    sess.commit()

print(f"🔗 player_news enlazados: {inserted}")


In [ ]:
# Conteo global
with engine.connect() as conn:
    n_players_news = conn.scalar(sa.text("SELECT COUNT(*) FROM player_news"))

In [ ]:
print(f"Hay {n_players_news:,} jugadores con noticas asociadas")

In [ ]:
# Top-10 jugadores con más artículos
top = pd.read_sql("""
    SELECT p.full_name, COUNT(*) AS n_news
    FROM player_news pn
    JOIN players p ON p.id = pn.player_id
    GROUP BY p.full_name
    ORDER BY n_news DESC
    LIMIT 10
""", engine)
display(top)

# Últimas 5 noticias para Rodri
rodri_news = pd.read_sql("""
    SELECT n.title, n.summary, n.published_at
    FROM player_news pn
    JOIN football_news n ON n.id = pn.news_id
    WHERE pn.player_id = 842
    ORDER BY n.published_at DESC
    LIMIT 5
""", engine)
display(rodri_news)